In [1]:
import pandas as pd
import time
from openai import OpenAI
import json
import regex
import tiktoken
import os
import re
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Say this is a test",
        }
    ],
    model="gpt-3.5-turbo",
)

In [3]:
base_path="/Users/sijiawu/Work/Thesis/Data"
temp=base_path+'/PDFs/RES/'
pt=base_path+"/gpt"

In [4]:
Merged=pd.read_excel(base_path+'/Combined/RES_M_sco_du.xlsx')
Merged.loc[Merged['journal']=='The Review of Economic Studies','journal']='RES'
Merged["ID"]=Merged["URL"].str.split("/").str[-1]
print(Merged.columns)
print(Merged.content_type.value_counts())

Index(['issue_url', 'ISSN', 'URL', 'journal', 'number', 'publisher', 'title',
       'urldate', 'volume', 'year', 'abstract', 'author', 'pages',
       'reviewed-author', 'uploaded', 'content_type', 'author_split',
       'title_10', 'type', 'authorsSCO', 'titleSCO', 'journalSCO', 'DOI',
       'affiliations', 'abstractSCO', 'citations', 'document type',
       'index keywords', 'author keywords', 'document_type', 'ID'],
      dtype='object')
Article      3340
MISC          718
Comment        60
Reply          15
Rejoinder       9
Name: content_type, dtype: int64


In [5]:
def num_tokens_from_messages(messages, model="gpt-4o-mini"):
#   """Returns the number of tokens used by a list of messages."""
  try:
      encoding = tiktoken.encoding_for_model(model)
  except KeyError:
      encoding = tiktoken.get_encoding("cl100k_base")
  if model == "gpt-4o-mini":  # note: future models may deviate from this
      num_tokens = 0
      for message in messages:
          num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
          for key, value in message.items():
              num_tokens += len(encoding.encode(value))
              if key == "name":  # if there's a name, the role is omitted
                  num_tokens += -1  # role is always required and always 1 token
      num_tokens += 2  # every reply is primed with <im_start>assistant
      return num_tokens
  else:
      raise NotImplementedError(f"""num_tokens_from_messages() is not presently implemented for model {model}.
  See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens.""")

def get_completion(prompt, model="gpt-4o-mini"):
    #gpt-3.5-turbo-16k
    messages = [{"role": "user", "content": prompt}]

    r_out = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,
        # request_timeout=5000
    )
    return r_out

def unravel(res):
    data=dict(res)
    data["choices"]=dict(data["choices"][0])
    data["usage"]=dict(data["usage"])
    data["choices"]["message"]=dict(data["choices"]["message"])
    data["usage"]['completion_tokens_details']=dict(data["usage"]['completion_tokens_details'])
    data['usage']['prompt_tokens_details']=dict(data['usage']['prompt_tokens_details'])
    return data

def open_json(filepath):
    with open(filepath) as json_data:
        d = json.load(json_data)
        json_data.close()
        return d  
    
def save_json(id, data):
    temp_dict={id: data}
    with open(base_path+'/gpt/'+id+'_chatgpt.json', 'w') as f:
        json.dump(temp_dict, f)

In [6]:
count=0
tks=0
temp=0
regex_error=[]
re_shard=[]
no_ref=[]
indivs=[]
abnormal=[]
apps=[]
lg=[]
incomplete=[]
eyes=[]
eyesi=[]
for i in Merged.index:
    res=None
    found=0
    count=count+1
    id=Merged.loc[i,'ID']

    if (Merged.loc[i, 'content_type']=='Review') | (Merged.loc[i,'content_type']=='MISC'):
        continue
    filename=base_path+'/gpt/'+id+'_chatgpt.json'
    filename_t=base_path+'/Tesseract/RES_Tesseract/'+id+"_tesseract.json"

    if os.path.exists(filename):
        print(str(count)+' '+id+" completed")
        continue

    if os.path.exists(filename_t)==False:
        print("tesseract file for "+ id +' not generated')
        re_shard.append(id)
        continue
    eyes.append(Merged.loc[i,'year'])

    data=open_json(filename_t)
    entry=Merged.iloc[i]
    print(entry["year"])
    
    refs=data[id]["references"]
    
    if "found" in refs.keys():
        found=1
        incomplete.append(id)
        eyesi.append(entry['year'])
    elif "raw" in refs.keys():
        print(str(id)+" has no references, check it")
        no_ref.append(id)
        continue
    else:
        if len(refs.keys())==0:
                no_ref.append(id)
                print('pdf not found for '+ id)
        else:
            print("error")
    
    if found==1:
        #response=[]
        #print(refs["found"].keys())
        #print(refs)
        pages=list(refs["found"].keys())
        pages_int = [int(numeric_string) for numeric_string in pages]
        pages_int.sort()
        print(pages_int)
        text=""
        
        for j in pages_int:
            text=text+refs["found"][str(j)][0]+"\nSpecPageDelimiter"
#         print(text)
        position=None
        position_a=None
        text=re.sub('\nthis content downloaded(?s:.*?)terms and conditions\n|\nthis content downloaded(?s:.*?)jstor.org/terms\n', "", text.lower(),re.M)
        
    
        try:
            position=regex.search('(^|\n)R(EFERENCES){e<=3}(\n| )', text.upper()).span(0)[0]
        except:
            print("Regex error: "+i)
            regex_error.append(i)
            continue
 
        try:
            position_a=regex.search('(^|\n)(APPENDIX){e<=3}(\n| )', text.upper()).span(0)[0]
            if position < position_a:
                apps.append(i)
                print("ref at "+str(position)+ ". app at "+ str(position_a) + " of "+str(len(text))+" in "+i)
            else:
                position_a=len(text)
        except:
            position_a=len(text)
        start=time.time()
        
        #prompt = 'given the following data. can you please format the references in the text into a .csv format with the following fields: authors, year, month, title, publisher, city, pages, the full reference. Please separate the author names with ";" as the delimiter if there are multiple authors. Please use quotes around text and "" for missing data.\n'+text[position:]
        #prompt = 'given the following data. can you please format the references in the text into a .csv format with the following fields: authors, year, month, title, publisher, pages, the full reference. Please separate the author names with "and" as the delimiter if there are multiple authors. Please use quotes around text and "" for missing data. Use ; as the delimiter.\n'+text[position:]
        #prompt = 'given the following data. can you please extract and format the references in the text into a json dictionary with the following fields: authors, year, month, title, publisher, city, pages, the full reference. Please separate the author names with ";" as the delimiter if there are multiple authors. Please use quotes around text and "" for missing data.\n'+text[position:]

        #prompt = 'please extract the references in the following data and format it in chicago referencing style.'+text[position:position_a].upper()
        #prompt = 'please extract the references in the following data and format it in harvard referencing style.'+text[position:position_a]
        #prompt = 'Given the following reference list. Please extract the following fields: authors, year, month, title, publisher, pages, and the full reference in Chicago referencing style. Please separate the author names with ";" as the delimiter if there are multiple authors. Please "NA" as a placeholder for missing data.\n'+text[position:position_a].upper()
        base_prompt='Given the following reference list, please extract the following fields of the reference into a json dictionary: authors, year, title, month, publisher, pages, and the full reference in Chicago referencing style. Please separate the author names with ";" as the delimiter if there are multiple authors. Please use "NA" as a placeholder for missing data.\n'
#        print(prompt)
        prompt = base_prompt+text[position:position_a].upper()
        toks=num_tokens_from_messages([{"role": "user", "content": prompt}])
        tks=tks+toks
        
        indivs.append(toks)
        temp=temp+1
        success=False
        if toks<3501:
            print(str(toks)+" standard")
            res=unravel(get_completion(prompt))
            if res["choices"]["finish_reason"]=="stop":
                save_json(id, res)
                success=True
            else:
                print("failed to return appropriate length")
                success=False
                
        if (toks>3500) | (success==False):
            print(str(toks)+" too long. Using big context model.")
            lg.append(id)
#             res=get_completion(prompt, "gpt-3.5-turbo-16k")
#             response[i]=res
            iter_text=text[position:position_a].split("\nspecpagedelimiter")
            print(len(iter_text))
            res={}
            res_count=0
            success_l=True
            for txt in iter_text:
                res[res_count]=unravel(get_completion(base_prompt+txt.upper()))
                if res[0]['choices']['finish_reason']=="length":
                    success_l = False
                    print("this is fucked up big")
                    break
                res_count+=1
            
            if success_l:
                save_json(id, res)
            # break
    print(toks)

1 rdz066 completed
2 rdz065 completed
3 rdz064 completed
4 rdz063 completed
5 rdz062 completed
6 rdz061 completed
7 rdz060 completed
8 rdz059 completed
9 rdz058 completed
10 rdz057 completed
11 rdz056 completed
12 rdz055 completed
13 rdz054 completed
14 rdz053 completed
15 rdz052 completed
16 rdz051 completed
17 rdz050 completed
18 rdz049 completed
19 rdz048 completed
20 rdz047 completed
21 rdz046 completed
22 rdz045 completed
23 rdz044 completed
24 rdz043 completed
25 rdz042 completed
26 rdz041 completed
27 rdz040 completed
28 rdz039 completed
29 rdz038 completed
30 rdz037 completed
31 rdz036 completed
32 rdz035 completed
33 rdz034 completed
34 rdz033 completed
35 rdz032 completed
36 rdz031 completed
37 rdz030 completed
38 rdz029 completed
39 rdz028 completed
40 rdz027 completed
41 rdz026 completed
42 rdz025 completed
43 rdz024 completed
44 rdz023 completed
45 rdz022 completed
46 rdz021 completed
47 rdz019 completed
48 rdz018 completed
49 rdz017 completed
50 rdz016 completed
51 rdz015